## Importing used libraries

In [1]:
# These three lines are necessary only if GemPy is not installed
import sys, os
sys.path.append('../..')
sys.path.append('../../../gempy/')

# Importing GemPy
import gempy as gp

# Importing aux libraries
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pn
import matplotlib
import theano
import qgrid
import pickle

# Imort SandBox
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN"
#import sandbox.sandbox as sb


In [2]:
# calibrationdata = sb.CalibrationData(file='my_calibration.json')
# kinect = sb.KinectV2(calibrationdata)
# projector = sb.Projector(calibrationdata)

### Initializing the model:

The first step to create a GemPy model is create a gempy.Model object that will contain all the other data structures and necessary functionality.

In addition for this example we will define a regular grid since the beginning. This is the grid where we will interpolate the 3D geological model. GemPy comes with an array of different grids for different pourposes as we will see below. For visualization usually a regular grid is the one that makes more sense.

In [3]:
geo_model = gp.create_model('Geological_Model1')
geo_model = gp.init_data(geo_model, extent= [0, 4000, 0, 2775, 200, 1200], resolution=[100, 10, 100])

Active grids: ['regular']


GemPy core code is written in Python. However for efficiency (and other reasons) most of heavy computations happend in optimize compile code, either C or CUDA for GPU. To do so, GemPy rely on the library theano. To guarantee maximum optimization theano requires to compile the code for every Python kernel. The compilation is done by calling the following line at any point (before computing the model):

In [4]:
gp.set_interpolator(geo_model, theano_optimizer='fast_run', verbose=[])

Compiling theano function...
Level of Optimization:  fast_run
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!


### Creating figure:

GemPy uses matplotlib and pyvista-vtk libraries for 2d and 3d visualization of the model respectively. One of the design decisions of GemPy is to allow real time construction of the model. What this means is that you can start adding input data and see in real time how the 3D surfaces evolve. Lets initialize the visualization windows.

The first one is the 2d figure. Just place the window where you can see it (maybe move the jupyter notebook to half screen and use the other half for the renderers).

In [5]:
#Visualization Widgets - Conflicts with bokeh visualization
from gempy.plot import visualization_2d_pro as vv
#from gempy.plot import vista

In [6]:
%matplotlib qt5

p2d = vv.Plot2D(geo_model)
p2d.create_figure((15, 8))


(<Figure size 3000x1600 with 0 Axes>, array([], shape=(0, 0), dtype=object))

#### Add model section

In the 2d renderer we can add several cross section of the model. In this case, for simplicity we are just adding one perpendicular to z.

In [7]:
# In this case perpendicular to the z axes
ax = p2d.add_section(direction='z', ax_pos=121)
#ax.imshow()

In [8]:
ax2 = p2d.add_section(direction='y', ax_pos=122)
ax2.set_xlim(geo_model.grid.regular_grid.extent[0], geo_model.grid.regular_grid.extent[1])
ax2.set_ylim(geo_model.grid.regular_grid.extent[4], geo_model.grid.regular_grid.extent[5])

(200.0, 1200.0)

#### Loading geological map image:

Remember that gempy is simply using matplotlib and therofe the ax object created above is a standard matplotlib axes. This allow to manipulate it freely. Lets load an image with the information of geological map

In [9]:
# Reading image
img = mpimg.imread('geological_model.png')
# Plotting it inplace
ax.imshow(img, origin='upper', alpha=.8, extent = (0, 4000, 0,2775))



## Building the model

Now that we have everything initialize we can start the construction of the geological model. 
### Cycle1:

#### Surfaces

GemPy is a surface based interpolator. This means that all the input data we add has to be refered to a surface. The surfaces always mark the bottom of a unit. By default GemPy surfaces are empty:

In [10]:
geo_model.surfaces

,surface,series,order_surfaces,color,id


We can create the first surfaces for the Cycle1 of the sedimentary layers:

In [11]:
geo_model.add_surfaces(['D','C','B', 'A'])

,surface,series,order_surfaces,color,id
0,D,Default series,1,#015482,1
1,C,Default series,2,#9f0052,2
2,B,Default series,3,#ffbe00,3
3,A,Default series,4,#728f02,4


Series is the object that contains the properties associated with each independent scalar field. The name by default is "Default series" but we can rename it and create new ones as we advance in the constructin of the model

In [12]:
geo_model.series

,order_series,BottomRelation
Default series,1,Erosion


In [13]:
geo_model.rename_series(['Cycle1'])

,order_series,BottomRelation
Cycle1,1,Erosion


Now we can start adding data. GemPy input data consist on surface points and orientations (perpendicular to the layers). The 2D plot gives you the X and Y coordinates when hovering the mouse over. We can add a surface point as follows:

In [14]:
#surface B
geo_model.add_surface_points(X=584, Y=285, Z=500, surface='B')
geo_model.add_surface_points(X=494, Y=696, Z=500, surface='B')
geo_model.add_surface_points(X=197, Y=1898, Z=500, surface='B')
geo_model.add_surface_points(X=473, Y=2180, Z=400, surface='B')
geo_model.add_surface_points(X=435, Y=2453, Z=400, surface='B')
#surface C
geo_model.add_surface_points(X=946, Y=188, Z=600, surface='C')
geo_model.add_surface_points(X=853, Y=661, Z=600, surface='C')
geo_model.add_surface_points(X=570, Y=1845, Z=600, surface='C')
geo_model.add_surface_points(X=832, Y=2132, Z=500, surface='C')
geo_model.add_surface_points(X=767, Y=2495, Z=500, surface='C')
#Surface D
geo_model.add_surface_points(X=967, Y=1638, Z=800, surface='D')
geo_model.add_surface_points(X=1095, Y=996, Z=800, surface='D')


,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
10,967.0,1638.0,800.0,0.569671,0.564361,0.543446,D,Cycle1,1,1,0.000001
11,1095.0,996.0,800.0,0.597413,0.425219,0.543446,D,Cycle1,1,1,0.000001
5,946.0,188.0,600.0,0.565120,0.250100,0.500100,C,Cycle1,2,1,0.000001
6,853.0,661.0,600.0,0.544963,0.352614,0.500100,C,Cycle1,2,1,0.000001
7,570.0,1845.0,600.0,0.483628,0.609224,0.500100,C,Cycle1,2,1,0.000001
8,832.0,2132.0,500.0,0.540412,0.671426,0.478427,C,Cycle1,2,1,0.000001
9,767.0,2495.0,500.0,0.526325,0.750100,0.478427,C,Cycle1,2,1,0.000001
0,584.0,285.0,500.0,0.486663,0.271123,0.478427,B,Cycle1,3,1,0.000001
1,494.0,696.0,500.0,0.467157,0.360200,0.478427,B,Cycle1,3,1,0.000001
2,197.0,1898.0,500.0,0.402787,0.620711,0.478427,B,Cycle1,3,1,0.000001


The minimum amount of data to interpolate anything in gempy is

a) 2 surface points per surface
b) One orientation per series.

Lets add an orientation for the first cycle:

In [15]:
# Adding orientation
geo_model.add_orientations(X=832, Y=2132, Z=500, surface='C', orientation = [98,17.88,1])

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth
0,832.0,2132.0,500.0,0.540412,0.671426,0.478427,0.304036,-0.04273,0.951702,17.88,98.0,1.0,C,Cycle1,2,1,0.01


In [16]:
# Plot in 2D
p2d.plot_data(ax, direction='z')
p2d.plot_data(ax2, direction='y', )


Now we have enough data for finally interpolate!

In [17]:
gp.compute_model(geo_model)

../../../gempy\gempy\core\solution.py:266: UserWarning: Attribute error. Using non masked marching cubesmarching_cubes_lewiner() got an unexpected keyword argument 'mask'.
  warnings.warn('Attribute error. Using non masked marching cubes' + str(e)+'.')



Lithology ids 
  [4. 4. 4. ... 1. 1. 1.] 

In [18]:
# In 2D

p2d.plot_contacts(ax2, direction='y', cell_number=5)
p2d.plot_lith(ax2, direction='y', cell_number=5)
# In 3D
#p3d.plot_surfaces()

0 3


In [19]:
gp.plot.plot_3D(geo_model)

holding... Use vtk.resume to go back to the interactive window


### Fault1 :
So far the model is simply a depositional unit. GemPy allows for unconformities and faults to build complex models. This input is given by categorical data. In general:

input data (surface points/ orientations) <belong to< surface <belong to< series

And series can be a fault---i.e. offset the rest of surface--- or not. We are going to show how to add a fault.

First we need to add a series:

In [20]:
geo_model.add_series(['Fault1'])

,order_series,BottomRelation
Cycle1,1,Erosion
Fault1,2,Erosion


In [21]:
geo_model.modify_order_series(1, 'Fault1')

,order_series,BottomRelation
Fault1,1,Erosion
Cycle1,2,Erosion


In [22]:
#geo_model.reorder_series(['Fault1', 'Cycle1'])

Then define that is a fault:

But we also need to add a new surface:

In [23]:
geo_model.add_surfaces(['F1'])

,surface,series,order_surfaces,color,id
0,D,Cycle1,1,#015482,1
1,C,Cycle1,2,#9f0052,2
2,B,Cycle1,3,#ffbe00,3
3,A,Cycle1,4,#728f02,4
4,F1,Cycle1,5,#443988,5


And finally assign the new surface to the new series/fault

In [24]:
gp.map_series_to_surfaces(geo_model, {'Fault1':'F1'})

,surface,series,order_surfaces,color,id
4,F1,Fault1,1,#443988,1
0,D,Cycle1,1,#015482,2
1,C,Cycle1,2,#9f0052,3
2,B,Cycle1,3,#ffbe00,4
3,A,Cycle1,4,#728f02,5


In [25]:
geo_model.set_is_fault('Fault1')

Fault colors changed. If you do not like this behavior, set change_color to False.


,isFault,isFinite
Fault1,True,False
Cycle1,False,False


Now we can just add input data as before (remember the minimum amount of input data to compute a model):

In [26]:
# Add input data of the fault
geo_model.add_surface_points(X=1203, Y=138, Z=600, surface='F1')
geo_model.add_surface_points(X=1250, Y=1653, Z=800, surface='F1')
#geo_model.add_surface_points(X=1280, Y=2525, Z=500, surface='F1')
#Add orientation
geo_model.add_orientations(X=1280, Y=2525, Z=500, surface='F1', orientation = [272,90,-1])

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth
1,1280.0,2525.0,500.0,0.613527,0.750100,0.479153,0.999391,-0.034899,9.999388e-13,90.00,272.0,-1.0,F1,Fault1,1,1,0.01
0,832.0,2132.0,500.0,0.519685,0.667779,0.479153,0.304036,-0.042730,9.517016e-01,17.88,98.0,1.0,C,Cycle1,3,2,0.01


In [27]:
# Compute
gp.compute_model(geo_model)

# Plot
p2d.plot_data(ax, direction='z')
p2d.plot_lith(ax2, cell_number=5)
#p3d.plot_structured_grid(opacity=.2, annotations = {2: 'surface1', 3:'surface2', 4:'surface3', 5:'basement'})

As you can see now instead of having dipping layers we have a sharp jump. But there is no information on the other side of the fault. That is because we now are going to add the information on the afected block. 

In [28]:
#surface B
geo_model.add_surface_points(X=1447, Y=2554, Z=500, surface='B')
geo_model.add_surface_points(X=1511, Y=2200, Z=500, surface='B')
geo_model.add_surface_points(X=1549, Y=629, Z=600, surface='B')
geo_model.add_surface_points(X=1630, Y=287, Z=600, surface='B')
#surface C
geo_model.add_surface_points(X=1891, Y=2063, Z=600, surface='C')
geo_model.add_surface_points(X=1605, Y=1846, Z=700, surface='C')
geo_model.add_surface_points(X=1306, Y=1641, Z=800, surface='C')
geo_model.add_surface_points(X=1476, Y=979, Z=800, surface='C')
geo_model.add_surface_points(X=1839, Y=962, Z=700, surface='C')
geo_model.add_surface_points(X=2185, Y=893, Z=600, surface='C')
geo_model.add_surface_points(X=2245, Y=547, Z=600, surface='C')
#Surface D
geo_model.add_surface_points(X=2809, Y=2567, Z=600, surface='D')
geo_model.add_surface_points(X=2843, Y=2448, Z=600, surface='D')
geo_model.add_surface_points(X=2873, Y=876, Z=700, surface='D')


,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
12,1203.0,138.0,600.0,0.533006,0.250100,0.500100,F1,Fault1,1,1,0.000001
13,1250.0,1653.0,800.0,0.542732,0.563635,0.541491,F1,Fault1,1,1,0.000001
10,967.0,1638.0,800.0,0.484165,0.560530,0.541491,D,Cycle1,2,2,0.000001
11,1095.0,996.0,800.0,0.510655,0.427666,0.541491,D,Cycle1,2,2,0.000001
25,2809.0,2567.0,600.0,0.865373,0.752790,0.500100,D,Cycle1,2,2,0.000001
26,2843.0,2448.0,600.0,0.872410,0.728163,0.500100,D,Cycle1,2,2,0.000001
27,2873.0,876.0,700.0,0.878618,0.402832,0.520795,D,Cycle1,2,2,0.000001
5,946.0,188.0,600.0,0.479819,0.260448,0.500100,C,Cycle1,3,2,0.000001
6,853.0,661.0,600.0,0.460572,0.358337,0.500100,C,Cycle1,3,2,0.000001
7,570.0,1845.0,600.0,0.402004,0.603370,0.500100,C,Cycle1,3,2,0.000001


In [30]:
# Compute
gp.compute_model(geo_model)

# Plot
p2d.remove(ax)
p2d.plot_data(ax, direction='z', cell_number=11)

p2d.remove(ax2)
p2d.plot_lith(ax2, cell_number=5)


Now all the first sequence is complete, with the deposition of some sedimentary layes and its posterior faulting

### Cycle2 :
For the second cycle, we have the disconformity of layer G on top of the old layers and the fault. This order is important to take into account for the modelling.
First we create a new serie with tis cycle:

In [31]:
geo_model.add_series(['Cycle2'])

,order_series,BottomRelation
Fault1,1,Fault
Cycle1,2,Erosion
Cycle2,3,Erosion


Now we need to organize the order of events from older in the bottom to youger in the top

In [40]:
geo_model.modify_order_series(1, 'Cycle2')

,order_series,BottomRelation
Cycle2,1,Erosion
Cycle1,2,Erosion
Fault1,3,Fault


In [41]:
geo_model.modify_order_series(2, 'Fault1')

,order_series,BottomRelation
Cycle2,1,Erosion
Fault1,2,Fault
Cycle1,3,Erosion


In [32]:
geo_model.reorder_series(['Cycle2','Fault1','Cycle1'])

,order_series,BottomRelation
Cycle2,1,Fault
Fault1,2,Erosion
Cycle1,3,Erosion


Now we see that the relation of the Fault and Cycle is switched. We need to fix this:

In [33]:
geo_model.set_is_fault('Fault1')
geo_model.set_is_fault('Cycle2',True, twofins=True, change_color=False)

Fault colors changed. If you do not like this behavior, set change_color to False.


,isFault,isFinite
Cycle2,False,False
Fault1,True,False
Cycle1,False,False


In [34]:
geo_model.series

,order_series,BottomRelation
Cycle2,1,Erosion
Fault1,2,Fault
Cycle1,3,Erosion


Now we can add the new surfaces, map the surfaces to the serie and assign the surface points and orientations. 

In [35]:
geo_model.add_surfaces(['G', 'H'])

,surface,series,order_surfaces,color,id
4,F1,Fault1,1,#443988,1
0,D,Cycle1,1,#015482,2
1,C,Cycle1,2,#9f0052,3
2,B,Cycle1,3,#ffbe00,4
3,A,Cycle1,4,#728f02,5
5,G,Cycle1,5,#ff3f20,6
6,H,Cycle1,6,#325916,7


In [36]:
geo_model.surfaces

,surface,series,order_surfaces,color,id
4,F1,Fault1,1,#443988,1
0,D,Cycle1,1,#015482,2
1,C,Cycle1,2,#9f0052,3
2,B,Cycle1,3,#ffbe00,4
3,A,Cycle1,4,#728f02,5
5,G,Cycle1,5,#ff3f20,6
6,H,Cycle1,6,#325916,7


In [37]:
geo_model.faults

,isFault,isFinite
Cycle2,False,False
Fault1,True,False
Cycle1,False,False


In [38]:
gp.map_series_to_surfaces(geo_model, {'Cycle2':['G','H']})

,surface,series,order_surfaces,color,id
5,G,Cycle2,1,#ff3f20,1
6,H,Cycle2,2,#325916,2
4,F1,Fault1,1,#443988,3
0,D,Cycle1,1,#015482,4
1,C,Cycle1,2,#9f0052,5
2,B,Cycle1,3,#ffbe00,6
3,A,Cycle1,4,#728f02,7


In [39]:
# Surface G
geo_model.add_surface_points(X=1012, Y=1493, Z=900, surface='G')
geo_model.add_surface_points(X=1002, Y=1224, Z=900, surface='G')
geo_model.add_surface_points(X=1996, Y=47, Z=800, surface='G')
geo_model.add_surface_points(X=300, Y=907, Z=700, surface='G')
#Surface H
geo_model.add_surface_points(X=3053, Y=727, Z=800, surface='G')
#Orientation
geo_model.add_orientations(X=1996, Y=47, Z=800, surface='G', orientation = [272,5.54,1])

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth
2,1996.0,47.0,800.0,0.565051,0.279512,0.526361,-0.096482,0.003369,9.953290e-01,5.54,272.0,1.0,G,Cycle2,1,1,0.01
1,1280.0,2525.0,500.0,0.439701,0.713335,0.473839,0.999391,-0.034899,9.999388e-13,90.00,272.0,-1.0,F1,Fault1,3,2,0.01
0,832.0,2132.0,500.0,0.361269,0.644533,0.473839,0.304036,-0.042730,9.517016e-01,17.88,98.0,1.0,C,Cycle1,5,3,0.01


In [44]:
# Compute
gp.compute_model(geo_model)

# Plot
p2d.remove(ax)
p2d.plot_data(ax, direction='z', cell_number=5, legend='force')

p2d.plot_lith(ax2, cell_number=5)

../../../gempy\gempy\core\solution.py:266: UserWarning: Attribute error. Using non masked marching cubesmarching_cubes_lewiner() got an unexpected keyword argument 'mask'.
  warnings.warn('Attribute error. Using non masked marching cubes' + str(e)+'.')


In [45]:
geo_model.surfaces

,surface,series,order_surfaces,color,id
5,G,Cycle2,1,#ff3f20,1
6,H,Cycle2,2,#325916,2
4,F1,Fault1,1,#443988,3
0,D,Cycle1,1,#015482,4
1,C,Cycle1,2,#9f0052,5
2,B,Cycle1,3,#ffbe00,6
3,A,Cycle1,4,#728f02,7


In [46]:
gp.plot.plot_3D(geo_model)

closing vtk


### Fault2: 
The model includes 2 different faultings and different ages. For this we are going to do the same steps as before to input the data:

In [47]:
geo_model.add_series(['Fault2'])

ValueError: new categories must not include old categories: {'Fault2'}

In [49]:
geo_model.reorder_series(['Fault2','Cycle2','Fault1','Cycle1'])

,order_series,BottomRelation
Fault2,1,Erosion
Cycle2,2,Fault
Fault1,3,Erosion
Cycle1,4,Erosion


In [51]:
geo_model.set_is_fault('Cycle2',toggle=True,twofins=True, change_color=False)

,isFault,isFinite
Fault2,True,False
Cycle2,False,False
Fault1,True,False
Cycle1,False,False


In [52]:
geo_model.series

,order_series,BottomRelation
Fault2,1,Fault
Cycle2,2,Erosion
Fault1,3,Fault
Cycle1,4,Erosion


In [53]:
geo_model.add_surfaces(['F2'])

,surface,series,order_surfaces,color,id
5,G,Cycle2,1,#ff3f20,1
6,H,Cycle2,2,#325916,2
4,F1,Fault1,1,#527682,3
0,D,Cycle1,1,#015482,4
1,C,Cycle1,2,#9f0052,5
2,B,Cycle1,3,#ffbe00,6
3,A,Cycle1,4,#728f02,7
7,F2,Cycle1,5,#5DA629,8


In [54]:
gp.map_series_to_surfaces(geo_model, {'Fault2':'F2'})

,surface,series,order_surfaces,color,id
7,F2,Fault2,1,#5DA629,1
5,G,Cycle2,1,#ff3f20,2
6,H,Cycle2,2,#325916,3
4,F1,Fault1,1,#527682,4
0,D,Cycle1,1,#015482,5
1,C,Cycle1,2,#9f0052,6
2,B,Cycle1,3,#ffbe00,7
3,A,Cycle1,4,#728f02,8


In [59]:
geo_model.set_is_fault(['Fault2'],twofins=True)
geo_model.set_is_fault(['Fault1'],twofins=True)

Fault colors changed. If you do not like this behavior, set change_color to False.
Fault colors changed. If you do not like this behavior, set change_color to False.


,isFault,isFinite
Fault2,True,False
Cycle2,False,False
Fault1,True,False
Cycle1,False,False


In [63]:
geo_model.surfaces.colors.reset_default_colors()

,surface,series,order_surfaces,color,id
7,F2,Fault2,1,#015482,1
5,G,Cycle2,1,#9f0052,2
6,H,Cycle2,2,#ffbe00,3
4,F1,Fault1,1,#728f02,4
0,D,Cycle1,1,#443988,5
1,C,Cycle1,2,#ff3f20,6
2,B,Cycle1,3,#325916,7
3,A,Cycle1,4,#5DA629,8


In [64]:
geo_model.faults.faults_relations_df

,Fault2,Cycle2,Fault1,Cycle1
Fault2,False,True,False,True
Cycle2,False,False,False,True
Fault1,False,False,False,True
Cycle1,False,False,False,False


In [65]:
geo_model.add_surface_points(X=3232, Y=178, Z=1000, surface='F2')
geo_model.add_surface_points(X=3132, Y=951, Z=700, surface='F2')
#geo_model.add_surface_points(X=2962, Y=2184, Z=700, surface='F2')

geo_model.add_orientations(X=3132, Y=951, Z=700, surface='F2', orientation = [95,90,1])

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,smooth
3,3132.0,951.0,700.0,0.733626,0.441451,0.500100,0.996195,-0.087156,1.000061e-12,90.00,95.0,1.0,F2,Fault2,1,1,0.01
2,1996.0,47.0,800.0,0.546476,0.292522,0.516574,-0.096482,0.003369,9.953290e-01,5.54,272.0,1.0,G,Cycle2,2,2,0.01
1,1280.0,2525.0,500.0,0.428518,0.700759,0.467151,0.999391,-0.034899,9.999388e-13,90.00,272.0,-1.0,F1,Fault1,4,3,0.01
0,832.0,2132.0,500.0,0.354713,0.636014,0.467151,0.304036,-0.042730,9.517016e-01,17.88,98.0,1.0,C,Cycle1,6,4,0.01


In [70]:
# Compute
gp.compute_model(geo_model)

# Plot
p2d.remove(ax)
p2d.plot_data(ax, direction='z', cell_number=5, legend='force')
p2d.plot_lith(ax, direction='z', cell_number=5)
p2d.plot_contacts(ax, direction='z',cell_number=5)

p2d.plot_lith(ax2, cell_number=5)

0 1
1 3
2 3
3 6


In [ ]:
gp.plot.plot_3D(geo_model)

KeyboardInterrupt: 

finally, we will add the information of the right side of fault2 to complete the model

In [67]:
geo_model.add_surface_points(X=3135, Y=1300, Z=700, surface='D')
geo_model.add_surface_points(X=3190, Y=969, Z=700, surface='D')

geo_model.add_surface_points(X=3031, Y=2725, Z=800, surface='G')
geo_model.add_surface_points(X=3018, Y=1990, Z=800, surface='G')
geo_model.add_surface_points(X=3194, Y=965, Z=700, surface='G')

geo_model.add_surface_points(X=3218, Y=1818, Z=890, surface='H')
geo_model.add_surface_points(X=3934, Y=1207, Z=810, surface='H')



,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
33,3232.0,178.0,1000.0,0.750100,0.314103,0.549523,F2,Fault2,1,1,0.000001
34,3132.0,951.0,700.0,0.733626,0.441451,0.500100,F2,Fault2,1,1,0.000001
28,1012.0,1493.0,900.0,0.384367,0.530743,0.533049,G,Cycle2,2,2,0.000001
29,1002.0,1224.0,900.0,0.382719,0.486426,0.533049,G,Cycle2,2,2,0.000001
30,1996.0,47.0,800.0,0.546476,0.292522,0.516574,G,Cycle2,2,2,0.000001
31,300.0,907.0,700.0,0.267069,0.434202,0.500100,G,Cycle2,2,2,0.000001
32,3053.0,727.0,800.0,0.720611,0.404548,0.516574,G,Cycle2,2,2,0.000001
37,3031.0,2725.0,800.0,0.716986,0.733708,0.516574,G,Cycle2,2,2,0.000001
38,3018.0,1990.0,800.0,0.714845,0.612621,0.516574,G,Cycle2,2,2,0.000001
39,3194.0,965.0,700.0,0.743840,0.443757,0.500100,G,Cycle2,2,2,0.000001


In [86]:
geo_model.faults.set_fault_relation(np.array([[False, True, False,  True],
       [False, False, False,  False],
       [False, False, False,  True],
       [False, False, False, False]]))

,Fault2,Cycle2,Fault1,Cycle1
Fault2,False,True,False,True
Cycle2,False,False,False,False
Fault1,False,False,False,True
Cycle1,False,False,False,False


In [85]:
geo_model.faults.faults_relations_df.values

array([[False, False, False,  True],
       [False, False, False,  True],
       [False, False, False,  True],
       [False, False, False, False]])

In [84]:
geo_model.faults.faults_relations_df

,Fault2,Cycle2,Fault1,Cycle1
Fault2,False,False,False,True
Cycle2,False,False,False,True
Fault1,False,False,False,True
Cycle1,False,False,False,False


In [105]:
# Compute
gp.compute_model(geo_model)

# Plot
p2d.plot_lith(ax, cell_number=5)
p2d.plot_contacts(ax, cell_number=5)


0 1
1 3
2 3
3 6


In [99]:
geo_model.interpolator.theano_graph.fault_relation.get_value()

array([[0, 1, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0]])

In [100]:
geo_model.surfaces

,surface,series,order_surfaces,color,id
7,F2,Fault2,1,#5DA629,1
6,H,Cycle2,1,#527682,2
5,G,Cycle2,2,#527682,3
4,F1,Fault1,1,#015482,4
0,D,Cycle1,1,#015482,5
1,C,Cycle1,2,#9f0052,6
2,B,Cycle1,3,#ffbe00,7
3,A,Cycle1,4,#728f02,8


In [101]:
geo_model.additional_data

values
Structure isLith                                                                  True
          isFault                                                                 True
          number faults                                                              2
          number surfaces                                                            7
          number series                                                              4
          number surfaces per series                                      [1, 2, 1, 3]
          len surfaces surface_points                           [2, 8, 2, 2, 7, 12, 9]
          len series surface_points                                     [2, 10, 2, 28]
          len series orientations                                         [1, 1, 1, 1]
Options   dtype                                                                float64
          output                                                               geology
          theano_optimizer                                                    fast_run
          device                                                                   cpu
          verbosity                                                                 []
Kriging   range                                                                4969.97
          $C_o$                                                                 588110
          drift equations                                                 [3, 3, 3, 3]
          nugget grad                                                             0.01
          nugget scalar                                                          1e-06
Rescaling rescaling factor                                                     7072.61
          centers                      [1965.1525054215153, 1386.0, 777.3283549481893]

In [104]:
geo_model.surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series,smooth
33,3232.000000,178.000000,1000.000000,0.694187,0.323343,0.508023,F2,Fault2,1,1,0.000001
34,3132.000000,951.000000,700.000000,0.679555,0.436450,0.464126,F2,Fault2,1,1,0.000001
40,3478.675040,1700.011926,1491.711195,0.730281,0.546047,0.579971,H,Cycle2,2,2,0.000001
41,3614.119650,844.929095,1403.474796,0.750100,0.420929,0.567060,H,Cycle2,2,2,0.000001
28,1012.000000,1493.000000,900.000000,0.369352,0.515756,0.493390,G,Cycle2,3,2,0.000001
29,1002.000000,1224.000000,900.000000,0.367889,0.476396,0.493390,G,Cycle2,3,2,0.000001
30,1996.000000,47.000000,800.000000,0.513333,0.304175,0.478758,G,Cycle2,3,2,0.000001
31,300.000000,907.000000,700.000000,0.265171,0.430012,0.464126,G,Cycle2,3,2,0.000001
32,3053.000000,727.000000,800.000000,0.667996,0.403674,0.478758,G,Cycle2,3,2,0.000001
37,3031.000000,2725.000000,800.000000,0.664777,0.696025,0.478758,G,Cycle2,3,2,0.000001


In [ ]:
gp.plot.plot_3D(geo_model)

## Iteractive DataFrame

### Activating Qgrid

Qgrid is only a gempy dependency. Therefore to use it, first we need to activate it in a given model by using:

In [47]:
gp.activate_interactive_df(geo_model)

This will create the interactive dataframes objects. This dataframes are tightly linked to the main dataframes of each data class.

#### Series

In [48]:
geo_model.qi.qgrid_se

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

#### Faults

In [49]:
geo_model.qi.qgrid_fa

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

#### surfaces

In [50]:
geo_model.qi.qgrid_fo

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### surface points

In [51]:
geo_model.qi.qgrid_in

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Orientations

In [52]:
geo_model.qi.qgrid_or

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Remember we are always changing the main df as well!

### Plot

In [53]:
# Compute
gp.compute_model(geo_model)

# Plot
p2d.plot_lith(ax, cell_number=5)
p2d.plot_contacts(ax, cell_number=5)
p3d.plot_surfaces()

0 1
1 3
2 3
3 6


,val
0,PolyData (0x2080572bd68)\n N Cells:\t10232\n ...
1,PolyData (0x2080572b2e8)\n N Cells:\t9704\n ...
2,PolyData (0x2080572b828)\n N Cells:\t6462\n ...
4,PolyData (0x2080572ba68)\n N Cells:\t2182\n ...
5,PolyData (0x2080555bdc8)\n N Cells:\t7228\n ...
6,PolyData (0x20808087c48)\n N Cells:\t7132\n ...
7,PolyData (0x208055c9168)\n N Cells:\t2752\n ...


### Sandbox ??

In [ ]:
gpsb=sb.GemPyModule(geo_model, calibrationdata, kinect, projector)

In [ ]:
gpsb.setup()

In [ ]:
gpsb.run()

In [ ]:
gpsb.stop() 

In [ ]:
fig=gp.plot.plot_section(geo_model,90, direction ='x')

fig.fig.axes[0].set_xlim(800,0)

